In [1]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict, concatenate_datasets

# ========== 加载数据 & 按语言单独划分后合并 ==========
language_dirs = ['de', 'en', 'nl']
base_dir = 'data/train'

splits = {'train': [], 'validation': [], 'test': []}

for lang in language_dirs:
    lang_dir = os.path.join(base_dir, lang)
    tsv = os.path.join(lang_dir, 'validated.tsv')
    clips = os.path.join(lang_dir, 'clips')
    
    if not os.path.exists(tsv):
        print(f"Missing {lang}/validated.tsv, skipping...")
        continue

    # 读入 TSV
    df = pd.read_csv(tsv, sep='\t')
    total_count = len(df)

    # 过滤掉无 sentence 或 path 的行
    df = df[df['sentence'].notnull() & df['path'].notnull()]
    df['audio_path'] = df['path'].map(lambda x: os.path.join(clips, x))
    df = df[df['audio_path'].map(os.path.exists)]
    df['lang'] = lang

    valid_count = len(df)

    print(f"[{lang}] 原始样本数: {total_count}, 有效样本数: {valid_count}")

    if valid_count < 10:
        print(f"Skipping {lang} due to too few samples.")
        continue

    # 构建 HuggingFace dataset 并切分
    lang_dataset = Dataset.from_pandas(df[['audio_path', 'sentence', 'lang']])
    ds_split = lang_dataset.train_test_split(test_size=0.1, seed=42)
    tmp = ds_split['train'].train_test_split(test_size=0.1, seed=42)

    splits['train'].append(tmp['train'])
    splits['validation'].append(tmp['test'])
    splits['test'].append(ds_split['test'])

# 合并所有语言
dataset = DatasetDict({
    'train': concatenate_datasets(splits['train']),
    'validation': concatenate_datasets(splits['validation']),
    'test': concatenate_datasets(splits['test']),
})


[de] 原始样本数: 12901, 有效样本数: 12901
[en] 原始样本数: 2203, 有效样本数: 2203
[nl] 原始样本数: 38917, 有效样本数: 38917


In [3]:
import os
import pandas as pd

language_dirs = ['de', 'en', 'fr']
base_dir = 'data/train'

print("包含 `path` 和 `sentence` 字段的 TSV 文件如下：\n")

for lang in language_dirs:
    lang_dir = os.path.join(base_dir, lang)

    if not os.path.exists(lang_dir):
        continue

    for file_name in os.listdir(lang_dir):
        if file_name.endswith('.tsv'):
            tsv_path = os.path.join(lang_dir, file_name)
            try:
                df = pd.read_csv(tsv_path, sep='\t', nrows=1)  # 只读一行，快
                has_path = 'path' in df.columns
                has_sentence = 'sentence' in df.columns

                if has_path and has_sentence:
                    print(f"[{lang}] {file_name} ✅")

            except Exception as e:
                print(f"[{lang}] {file_name} 读取失败：{e}")


包含 `path` 和 `sentence` 字段的 TSV 文件如下：

[de] invalidated.tsv ✅
[de] other.tsv ✅
[de] validated.tsv ✅
[en] invalidated.tsv ✅
[en] other.tsv ✅
[en] validated.tsv ✅
[fr] invalidated.tsv ✅
[fr] other.tsv ✅
[fr] validated.tsv ✅


In [ ]:
import os
import shutil
import pandas as pd
from pathlib import Path

import random
random.seed(42)

language_dirs = ['de', 'en', 'fr']
source_base = 'data/train'
target_base = 'data1/train'
sample_size = 100

for lang in language_dirs:
    src_dir = os.path.join(source_base, lang)
    tsv_path = os.path.join(src_dir, 'validated.tsv')
    clips_dir = os.path.join(src_dir, 'clips')

    if not os.path.exists(tsv_path):
        print(f"[{lang}] Missing TSV, skipping.")
        continue

    df = pd.read_csv(tsv_path, sep='\t')
    df = df[df['sentence'].notnull() & df['path'].notnull()]
    df['audio_path'] = df['path'].map(lambda x: os.path.join(clips_dir, x))
    df = df[df['audio_path'].map(os.path.exists)]

    if len(df) < sample_size:
        print(f"[{lang}] Too few valid samples ({len(df)}), skipping.")
        continue

    df_sampled = df.sample(n=sample_size, random_state=42).copy()

    # 复制音频和写 TSV
    target_lang_dir = os.path.join(target_base, lang)
    target_clips_dir = os.path.join(target_lang_dir, 'clips')
    os.makedirs(target_clips_dir, exist_ok=True)

    for _, row in df_sampled.iterrows():
        src_audio_path = row['audio_path']
        tgt_audio_path = os.path.join(target_clips_dir, os.path.basename(src_audio_path))
        shutil.copy2(src_audio_path, tgt_audio_path)

    # 更新 path 字段（相对路径）
    df_sampled['path'] = df_sampled['path'].map(lambda x: os.path.basename(x))

    # 写入新的 validated.tsv，只保留原始字段
    tsv_columns = [col for col in df.columns if col in df_sampled.columns]
    tsv_out = os.path.join(target_lang_dir, 'validated.tsv')
    df_sampled[tsv_columns].to_csv(tsv_out, sep='\t', index=False)

    print(f"[{lang}] Copied 1000 samples to {target_lang_dir}")


[de] Copied 1000 samples to data1/train/de


In [4]:
import os
import shutil
import pandas as pd
from pathlib import Path

import random
random.seed(42)

language_dirs = ['es', 'ja', 'ru','tr']
source_base = 'data/zero_shot'
target_base = 'data1/zero_shot'
sample_size = 100

for lang in language_dirs:
    src_dir = os.path.join(source_base, lang)
    tsv_path = os.path.join(src_dir, 'validated.tsv')
    clips_dir = os.path.join(src_dir, 'clips')

    if not os.path.exists(tsv_path):
        print(f"[{lang}] Missing TSV, skipping.")
        continue

    df = pd.read_csv(tsv_path, sep='\t')
    df = df[df['sentence'].notnull() & df['path'].notnull()]
    df['audio_path'] = df['path'].map(lambda x: os.path.join(clips_dir, x))
    df = df[df['audio_path'].map(os.path.exists)]

    if len(df) < sample_size:
        print(f"[{lang}] Too few valid samples ({len(df)}), skipping.")
        continue

    df_sampled = df.sample(n=sample_size, random_state=42).copy()

    # 复制音频和写 TSV
    target_lang_dir = os.path.join(target_base, lang)
    target_clips_dir = os.path.join(target_lang_dir, 'clips')
    os.makedirs(target_clips_dir, exist_ok=True)

    for _, row in df_sampled.iterrows():
        src_audio_path = row['audio_path']
        tgt_audio_path = os.path.join(target_clips_dir, os.path.basename(src_audio_path))
        shutil.copy2(src_audio_path, tgt_audio_path)

    # 更新 path 字段（相对路径）
    df_sampled['path'] = df_sampled['path'].map(lambda x: os.path.basename(x))

    # 写入新的 validated.tsv，只保留原始字段
    tsv_columns = [col for col in df.columns if col in df_sampled.columns]
    tsv_out = os.path.join(target_lang_dir, 'validated.tsv')
    df_sampled[tsv_columns].to_csv(tsv_out, sep='\t', index=False)

    print(f"[{lang}] Copied 1000 samples to {target_lang_dir}")


[es] Too few valid samples (5), skipping.
[ja] Too few valid samples (482), skipping.
[ru] Too few valid samples (349), skipping.
[tr] Copied 1000 samples to data1/zero_shot/tr


In [3]:
import os
import pandas as pd
import soundfile as sf

language_dirs = ['de', 'en', 'nl']
target_base = 'data/train'

for lang in language_dirs:
    tsv_path = os.path.join(target_base, lang, 'validated.tsv')
    clips_dir = os.path.join(target_base, lang, 'clips')

    if not os.path.exists(tsv_path):
        print(f"[{lang}] Missing TSV file, skipping.")
        continue

    df = pd.read_csv(tsv_path, sep='\t')
    # 取 path 字段，构造音频完整路径
    audio_paths = df['path'].map(lambda x: os.path.join(clips_dir, x))

    total_duration = 0.0
    for audio_path in audio_paths:
        try:
            with sf.SoundFile(audio_path) as f:
                frames = f.frames
                samplerate = f.samplerate
                duration = frames / samplerate
                total_duration += duration
        except Exception as e:
            print(f"Error reading {audio_path}: {e}")

    print(f"[{lang}] Total duration of {len(audio_paths)} samples: {total_duration:.2f} seconds")


[de] Total duration of 12901 samples: 69881.33 seconds
[en] Total duration of 2203 samples: 12205.94 seconds
[nl] Total duration of 38917 samples: 152308.21 seconds
